In [4]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [5]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [6]:
from __future__ import print_function

j_st = 0
j_en = 100
ncname = 'OmA_horizon_2015_newalg_' + str(j_st) + '_' + str(j_en) + '.nc'

from numpy import *
from scipy import *
import netCDF4 as nc
import numpy as np
import scipy as sp
import seawater
import datetime as dt
""
from salishsea_tools import (
    nc_tools,
    viz_tools, 
    geo_tools,
    tidetools
)
from salishsea_tools import visualisations as vis
import matplotlib.pyplot as plt
#matplotlib.use('Agg')
import matplotlib.patches as patches
plt.style.use('seaborn-whitegrid')
import netCDF4 as nc

import cmocean as cm
import glob
import sys
sys.path.append('/data/tjarniko/mocsy')
sys.path.append('/data/tjarniko/MEOPAR/at3/notebooks/carbon_dev/CCCmaDEV/CCCma_src')
sys.path.append('/data/tjarniko/MEOPAR/tools/SalishSeaTools/salishsea_tools/')
import mocsy
import river_201702 as rv
import mocsy
import CCCma
import CCCma_stations as cs
from matplotlib import reload
import arrow
import gsw
import datetime as dt
import timeit
start_time = timeit.default_timer()
print(start_time)

11660454.9081832


11660758.789597794
0
250
[1.7881370782852173 1.7780309915542603 1.7670657634735107
 1.750502109527588 1.7357971668243408 1.7191379070281982
 1.7043709754943848 1.6913459300994873 1.679276943206787 1.668192982673645
 1.6586180925369263 1.6502357721328735 1.6402058601379395
 1.630528211593628 1.6208127737045288 1.6087963581085205
 1.5997321605682373 1.586973786354065 1.5678389072418213
 1.5471361875534058 1.5159765481948853 1.4519970417022705
 1.3660722970962524 1.2192333936691284 1.0521870851516724
 0.9292067289352417 0.8824480175971985 0.8594037294387817 -- -- -- -- --
 -- -- -- -- -- -- --]
nan
[1.8783012628555298 1.8677197694778442 1.8602780103683472
 1.8449677228927612 1.8323917388916016 1.8113348484039307
 1.7831604480743408 1.767972469329834 1.75031316280365 1.736624836921692
 1.7255604267120361 1.7174897193908691 1.7091840505599976
 1.7005614042282104 1.6916892528533936 1.6788759231567383
 1.663877010345459 1.6477618217468262 1.6275854110717773 1.601494312286377
 1.56287086009979

In [42]:
def is_empty(any_structure):
    if any_structure:
        print('Structure is not empty.')
        return False
    else:
        print('Structure is empty.')
        return True

11661518.414532248
day is
0
j is
250
[1.7881370782852173 1.7780309915542603 1.7670657634735107
 1.750502109527588 1.7357971668243408 1.7191379070281982
 1.7043709754943848 1.6913459300994873 1.679276943206787 1.668192982673645
 1.6586180925369263 1.6502357721328735 1.6402058601379395
 1.630528211593628 1.6208127737045288 1.6087963581085205
 1.5997321605682373 1.586973786354065 1.5678389072418213
 1.5471361875534058 1.5159765481948853 1.4519970417022705
 1.3660722970962524 1.2192333936691284 1.0521870851516724
 0.9292067289352417 0.8824480175971985 0.8594037294387817 -- -- -- -- --
 -- -- -- -- -- -- --]
(array([25, 26, 27]),)
nan
(array([25, 26, 27]),)
nan
0.007810546085238457
0.008695444092154503


In [75]:
def find_depth(dp,prof):
    #finds saturation horizon given a profile and corresponding depths
    first_proper_undersat = np.nan
    depth_undersat = np.nan    
    dummy_var = 0
    #tot masked
    if prof.mask.all():
        dummy_var = 0
        depth_undersat = np.nan
        #print('masks all around!')
    elif np.ma.min(prof) >1:
        depth_undersat = np.nan
        #print('saturated column')
    elif np.ma.max(prof) <1:
        depth_undersat = 0
        #print('undersat to surface')        
    else:
        t_ind = np.where(prof<1)
        t_indar = t_ind[0][0]
        t_indss = np.where(prof>=1)
        t_indsssar = t_indss[0][0]
        if t_indar.size == 0:
            dummy_var = 0
        else:
            if (t_indar.size != 0) & (t_indsssar.size == 0):
                depth_undersat = 0
                first_proper_undersat = 0
                dummy_var = 0
                #print('undersat to surface!')
                max_supsat = np.nan
            else:    
                max_supsat = np.max(t_indsssar)    
                try:
                    first_proper_undersat = np.min(t_indar[t_indar>max_supsat])
                except:
                    dummy_var = 0
                    #print("An exception occurred")
                if first_proper_undersat == 0:
                    depth_undersat = dp[0]
                if np.isnan(first_proper_undersat):
                    dummy_var = 0
                    #print('saturated watercolumn!')
                else:
                    depth_undersat = (dp[first_proper_undersat]+dp[first_proper_undersat-1])/2
    return depth_undersat

In [77]:
 
OmA = nc.Dataset('OmA_2015.nc')
#switch BR and PI
BR_omA = OmA['model_output']['OmAr_pi']
PI_omA = OmA['model_output']['OmAr_br']

t_nc = nc.Dataset('/results2/SalishSea/nowcast-green.201806/01jan18/SalishSea_1h_20180101_20180101_grid_T.nc')
zlevels = (t_nc['deptht'][:])


import timeit
start_time = timeit.default_timer()
print(start_time)

oma_d_PI = np.zeros([365,898,398])
oma_d_BR = np.zeros([365,898,398])

def find_depth(dp,prof):
    #finds saturation horizon given a profile and corresponding depths
    first_proper_undersat = np.nan
    depth_undersat = np.nan    
    dummy_var = 0
    #tot masked
    if prof.mask.all():
        dummy_var = 0
        depth_undersat = np.nan
        #print('masks all around!')
    elif np.ma.min(prof) >1:
        depth_undersat = np.nan
        #print('saturated column')
    elif np.ma.max(prof) <1:
        depth_undersat = 0
        #print('undersat to surface')        
    else:
        t_ind = np.where(prof<1)
        t_indar = t_ind[0][0]
        t_indss = np.where(prof>=1)
        t_indsssar = t_indss[0][0]
        if t_indar.size == 0:
            dummy_var = 0
        else:
            if (t_indar.size != 0) & (t_indsssar.size == 0):
                depth_undersat = 0
                first_proper_undersat = 0
                dummy_var = 0
                #print('undersat to surface!')
                max_supsat = np.nan
            else:    
                max_supsat = np.max(t_indsssar)    
                try:
                    first_proper_undersat = np.min(t_indar[t_indar>max_supsat])
                except:
                    dummy_var = 0
                    #print("An exception occurred")
                if first_proper_undersat == 0:
                    depth_undersat = dp[0]
                if np.isnan(first_proper_undersat):
                    dummy_var = 0
                    #print('saturated watercolumn!')
                else:
                    depth_undersat = (dp[first_proper_undersat]+dp[first_proper_undersat-1])/2
    return depth_undersat

for day in range(0,1):
    print('day is')
    print(day)
    for j in range(250,251):
        for i in range(0,398):
            OmA_BR_test = BR_omA[day,:,j,i]
            OmA_PI_test = PI_omA[day,:,j,i]

            oma_dep_BR = find_depth(zlevels,OmA_BR_test)
            oma_dep_PI = find_depth(zlevels,OmA_PI_test)
            oma_d_PI[day,j,i] = oma_dep_PI
            oma_d_BR[day,j,i] = oma_dep_BR
            if (i == 250) & (j%10 == 0):
                print('BR depth: '+str(oma_dep_BR), 'PI depth: '+str(oma_dep_PI))
elapsed = timeit.default_timer() - start_time
print(elapsed)
    

# f = nc.Dataset(ncname,'w', format='NETCDF4') #'w' stands for write
# g = f.createGroup('model_output')
# #g.createDimension('days', len(3))
# g.createDimension('days', 365)
# g.createDimension('ys', 898)
# g.createDimension('xs', 398)
# ts = g.createVariable('OmArHORIZON_pi','f4',('days','ys','xs'))
# ts[:] = oma_d_PI
# ts2 = g.createVariable('OmArHORIZON_br','f4',('days','ys','xs'))
# ts2[:] = oma_d_BR
# f.close()

elapsed = timeit.default_timer() - start_time
print(elapsed)

11662596.374663267
day is
0
250
BR depth: 67.53495788574219 PI depth: 67.53495788574219
0.7543164398521185
0.7546494472771883
